1. Construção da ABT

1.1 unificar tabelas

In [0]:
from pyspark.sql import SparkSession

orders = spark.sql("""select *
                   , year(order_timestamp_local) as ano
                   , month(order_timestamp_local) as mes
                   , concat(substring(order_timestamp_local,1,8),'01') as anomes  
                   from ifood.ifood_orders""")

orders.display()

In [0]:
segmentation = spark.sql("select * from ifood.ifood_customer_segmentation")

In [0]:
segmentation.display()

In [0]:

left_join = (segmentation.join(orders, ( segmentation["customer_id"] == orders["customer_id"]) & (segmentation["segmentation_month"] == orders["anomes"]), "left" )).drop(orders["customer_id"])


#left_join = (segmentation.join(orders, on = "customer_id",(segmentation["customer_id"] == orders["customer_id"]) & (segmentation["segmentation_month"] == orders["anomes"]), "left" ))


In [0]:
left_join.display()

In [0]:
ABT = left_join[["customer_id","top_city","last_nps","preferred_shift_bucket_description","merchant_variety_bucket","preferred_dishes",
           "customer_lifetime_months","order_shift","delivery_type","device_type","merchant_dish_type","payment_method","promo_is_promotion","order_date_local"]]

In [0]:
ABT_1 = ABT.filter(ABT.order_shift.isNotNull()).toPandas()

In [0]:
ABT_1.display()

In [0]:
temp =[]
for c in ABT_1.merchant_dish_type:
    if c == 'Comida Japonesa':
        temp.append(1)
    else:
        temp.append(0)
ABT_1['target'] = temp     

In [0]:
ABT_1.display()

In [0]:
!pip install pycaret

In [0]:
!pip install markupsafe == 21.2.4

In [0]:
import numpy as np
import pandas as pd

In [0]:
df_train = ABT_1.query('order_date_local < "2019-10-31"')
df_oot = ABT_1.query('order_date_local > "2019-10-31"')

In [0]:
key_vars = ['customer_id']
num_vars = ['customer_lifetime_months', 'promo_is_promotion','customer_lifetime_months']
cat_vars = ['top_city','last_nps','preferred_shift_bucket_description','merchant_variety_bucket','preferred_dishes','customer_lifetime_months','order_shift',
           'delivery_type','device_type','merchant_dish_type','payment_method']
target = 'target'

features = cat_vars + num_vars

In [0]:
ABT_1['target'].value_counts()

In [0]:
from pycaret.classification import *

In [0]:
exp = setup(data=df_train, test_data=df_oot, target=target,ignore_features=key_vars, categorical_features=cat_vars, numeric_features=num_vars, session_id=42)

In [0]:
get_config('X')

In [0]:
get_config('prep_pipe')

In [0]:
best_model = compare_models(fold=5, sort='auc')